In [3]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.applications import VGG19
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator


In [8]:
dataset_dir = "train"
open_eye_images = []
closed_eye_images = []

In [9]:
for image_filename in os.listdir(dataset_dir):
    if image_filename.startswith("Open"):
        # Load the image and preprocess it.
        image = cv2.imread(os.path.join(dataset_dir, image_filename, cv2.IMREAD_GRAYSCALE))
        if image is not None:  # Check if the image was loaded successfully.
            image = cv2.resize(image, (224, 224))  # Resize to the desired input size.
            image = image / 255.0  # Normalize the pixel values to [0, 1].
            open_eye_images.append(image)
        else:
            print(f"Failed to load image: {image_filename}")
    elif image_filename.startswith("Closed"):
        # Load the image and preprocess it.
        image = cv2.imread(os.path.join(dataset_dir, image_filename, cv2.IMREAD_GRAYSCALE))
        if image is not None:  # Check if the image was loaded successfully.
            image = cv2.resize(image, (224, 224))
            image = image / 255.0
            closed_eye_images.append(image)
        else:
            print(f"Failed to load image: {image_filename}")

TypeError: join() argument must be str, bytes, or os.PathLike object, not 'int'

In [ ]:
open_eye_data = np.array(open_eye_images)
closed_eye_data = np.array(closed_eye_images)


open_eye_labels = np.ones(open_eye_data.shape[0])
closed_eye_labels = np.zeros(closed_eye_data.shape[0])


data = np.vstack((open_eye_data, closed_eye_data))
labels = np.concatenate((open_eye_labels, closed_eye_labels))

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

In [ ]:
def build_drowsiness_detection_model(input_shape):
    base_model = VGG19(weights='imagenet', include_top=False, input_shape=input_shape)
    
    model = Sequential()
    model.add(base_model)
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    return model

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=20,     # Randomly rotate the images by up to 20 degrees.
    width_shift_range=0.2, # Randomly shift the width of the images.
    height_shift_range=0.2,# Randomly shift the height of the images.
    shear_range=0.2,       # Apply shear transformations.
    zoom_range=0.2,        # Randomly zoom in/out.
    horizontal_flip=True,  # Randomly flip images horizontally.
    vertical_flip=False,   # Do not flip vertically (usually not relevant for open/closed eyes).
    fill_mode='nearest'    # Fill empty pixels using the nearest-neighbor strategy.
)

# Apply data augmentation to your dataset (assuming you already have data and labels).
augmented_data = []
augmented_labels = []
batch_size = 32  # Adjust as needed.

for batch in datagen.flow(data, labels, batch_size=batch_size):
    augmented_data.extend(batch[0])
    augmented_labels.extend(batch[1])
    
    if len(augmented_data) >= len(data):
        break

# Convert the augmented data and labels to NumPy arrays.
augmented_data = np.array(augmented_data)
augmented_labels = np.array(augmented_labels)

In [ ]:
# Compile the model.
input_shape = (224, 224, 3)  # Adjust this to your image size
model = build_drowsiness_detection_model(input_shape)
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model using your training data.
model.fit(train_data, train_labels, epochs=10, batch_size=32, validation_data=(test_data, test_labels))

# Evaluate the model on the testing data.
accuracy, precision = model.evaluate(test_data, test_labels)

# Save the model for future use.
model.save('drowsiness_detection_model.h5')